### Coocurence profile

In [1]:
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.Draw import IPythonConsole
import pandas as pd
from rdkit.Chem import rdMolDescriptors as rdescriptors
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sys    
import pubchempy as pcp
import warnings
warnings.filterwarnings('ignore')
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*')
from sklearn.utils import shuffle
import sys
sys.path.insert(0,"./../fip3")
from fip.profiles import CooccurrenceProfile
from fip import profiles
import re
from itertools import chain
from ast import literal_eval

In [49]:
#Loading zinc database from csv 
suppl_csv = pd.read_csv('../data/ZINCECFP.csv', delimiter = ' ')
suppl_csv

,Zinc_id,Smiles,MW,logP,Inchi,Inchi_s,Name,ECFP,Bitset
0,1589384,N=C(N)NCC[C@H](N)C(=O)O,160.096026,-1.72853,IFPQOXNWLSRZKX-VKHMYHEASA-N,IFPQOXNWLSRZKX,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[1, 80, 140, 197, 389, 623, 650, 667, 739, 786..."
1,83822513,CN(C)CC(=O)NCC(=O)O,160.084792,-1.25120,HQFNONZTUQSPJS-UHFFFAOYSA-N,HQFNONZTUQSPJS,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[41, 80, 197, 339, 389, 390, 650, 652, 807, 88..."
2,214763687,O=C(O)CN1CCCNS1(=O)=O,194.036128,-1.38890,KCIDTUHJDPZBTQ-UHFFFAOYSA-N,KCIDTUHJDPZBTQ,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 187, 190, 207, 305, 307, 389, 469, 650, 8..."
3,306392345,O=C(O)[C@@H]1C[C@H]2O[C@H]1[C@H](O)[C@H]2O,174.052823,-1.41990,ZZTJOHOETCDWML-VFUOTHLCSA-N,ZZTJOHOETCDWML,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[209, 221, 285, 389, 650, 656, 807, 926, 1017,..."
4,85343607,N[C@@H]1N=C2N=C(O)N=C2C(=O)N1,167.044324,-1.87440,OSXQHYVRCFCLQV-SCSAIBSYSA-N,OSXQHYVRCFCLQV,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[15, 106, 301, 314, 378, 584, 650, 664, 724, 7..."
...,...,...,...,...,...,...,...,...,...
379007,97986502,Cc1ccc(C(=O)N(c2ccc(N(C(=O)c3ccc(C)cc3)S(=O)(=...,636.051720,6.49784,KYPDAVIHIYTGQW-UHFFFAOYSA-N,KYPDAVIHIYTGQW,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[14, 234, 235, 241, 350, 383, 517, 587, 650, 6..."
379008,97948800,CCc1c(-c2ccc(OC)c(OC)c2)nn(-c2nc(-c3ccc(C)cc3)...,604.229740,7.58732,AWQWFNRTEBAZJO-UHFFFAOYSA-N,AWQWFNRTEBAZJO,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 94, 114, 121, 140, 208, 235, 249, 289, 29..."
379009,54274624,Cc1cc(N2CCN(CCOc3cccc4ccccc34)CC2)n2nc(C)c(-c3...,511.213888,6.02074,HAHVCBFPDRUVIL-UHFFFAOYSA-N,HAHVCBFPDRUVIL,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[13, 74, 80, 92, 140, 212, 216, 262, 296, 378,..."
379010,2093448,CCOc1cc(CNc2cc(Cl)ccc2C)cc(I)c1OCc1ccccc1Cl,541.007232,7.49622,IYFPBZIIJDRXMB-UHFFFAOYSA-N,IYFPBZIIJDRXMB,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 99, 165, 216, 237, 255, 294, 295, 333, 36..."


In [ ]:
suppl_csv= suppl_csv.sample(frac=1).reset_index(drop=True)

In [ ]:
suppl_csv

In [ ]:
suppl_csv.to_csv('../data/CP1/ZINC/ZINCrandom.csv', sep=' ', index = False)

In [46]:
suppl_csv = pd.read_csv('../data/CP1/ZINC/ZINCrandom.csv', delimiter = ' ')

In [47]:
df_split = np.array_split(suppl_csv, 10)

In [48]:
df_split[0]

,Zinc_id,Smiles,MW,logP,Inchi,Inchi_s,Name,ECFP,Bitset
0,8432508,Cc1ccc(-n2c(=O)c3c(n4c(SCCc5ccccc5)nnc24)-c2cc...,532.229683,6.94842,DPUIXUVMXJXCJR-UHFFFAOYSA-N,DPUIXUVMXJXCJR,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[2, 58, 74, 80, 105, 116, 153, 233, 235, 289, ..."
1,2325339,Cc1ccc(-n2c(COc3ccccc3)nnc2SCC(=O)Nc2ccc3c(c2)...,488.151826,4.65672,HVIYVAXZIXJRQC-UHFFFAOYSA-N,HVIYVAXZIXJRQC,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[41, 58, 80, 102, 116, 127, 184, 191, 233, 235..."
2,408636482,COCCNS(=O)(=O)c1ccc(OCC(=O)N2CCC(C(N)=O)CC2)c(...,433.107434,0.36740,RAILYSNQBZZKMQ-UHFFFAOYSA-N,RAILYSNQBZZKMQ,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[13, 80, 94, 111, 140, 198, 222, 279, 317, 319..."
3,97995285,Cc1ccc(S(=O)(=O)n2nc(NCc3cccc(Br)c3)nc2NCc2ccc...,588.978270,5.57272,FFUQFTXMHIMSHO-UHFFFAOYSA-N,FFUQFTXMHIMSHO,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 82, 92, 157, 235, 255, 319, 350, 374, 378..."
4,408561088,CCn1c(-c2ccccc2)c(C)s/c1=N\C(=O)c1ccccc1Br,400.024496,5.04852,GMWZBJMZXGQSQF-VZCXRCSSSA-N,GMWZBJMZXGQSQF,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[30, 34, 57, 80, 162, 289, 294, 308, 313, 351,..."
...,...,...,...,...,...,...,...,...,...
37897,97995451,Cc1ccc(Cl)cc1N1CCN(C(=O)CN(c2ccc(Cl)c(Cl)c2)S(...,551.060396,5.49932,UEZQAXBXQIVHHC-UHFFFAOYSA-N,UEZQAXBXQIVHHC,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[24, 80, 165, 216, 231, 319, 350, 365, 366, 38..."
37898,408595552,COc1ccc(S(=O)(=O)N2CCC[C@H](C(=O)NCCCN3CCOCC3)...,425.198442,0.93440,SERMPMMEGHANCD-KRWDZBQOSA-N,SERMPMMEGHANCD,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 93, 197, 221, 319, 322, 350, 374, 446, 52..."
37899,408959652,Cc1ccc(CN2C(=O)S/C(=C\c3cccn3-c3ccc(N(C)C)cc3)...,417.151098,5.08822,CPCJMIHIVOAJEG-JCMHNJIXSA-N,CPCJMIHIVOAJEG,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[74, 80, 95, 233, 235, 273, 281, 310, 314, 475..."
37900,252489562,CC(C)(C)C(=O)N1CC[C@H](Oc2cccc3cccnc23)C1,298.168128,3.26060,JTLGIMMFHQVTQX-AWEZNQCLSA-N,JTLGIMMFHQVTQX,ZINC,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[114, 119, 145, 270, 378, 392, 448, 449, 524, ..."
